https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [ ]:
%pip install git+https://github.com/piskvorky/gensim.git@develop

In [ ]:
!pip install pandas --user
!pip install gensim --user
!pip install scikit-learn --user
pip3 install -U scikit-learn scipy matplotlib

In [ ]:
!pip install scikit-learn --user

In [ ]:
!pip install gensim==3.4.0
!pip install smart_open==1.9.0

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import smart_open
from gensim.models.keyedvectors import KeyedVectors

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
path = 'GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
# Cargar el dataset
df = pd.read_csv('career_prediction_dataset.csv')
print(df)

In [ ]:
categories = {
    "Healthcare and Medicine": [
        "Nurse", "Physician", "Pharmacist", "Physical Therapist", 
        "Dental Hygienist", "Pediatric Nurse", "Chiropractor", 
        "Radiologic Technologist", "Speech Therapist", "Occupational Therapist", 
        "Rehabilitation Counselor", "Pediatrician", "Genetic Counselor", 
        "Forensic Psychologist", "Public Health Analyst"
    ],
    "Engineering and Technology": [
        "Software Developer", "IT Support Specialist", "Biomedical Engineer", 
        "Data Analyst", "Industrial Engineer", "Mechanical Engineer", 
        "Environmental Engineer", "Electrical Engineer", "Aerospace Engineer", 
        "Mechanical Designer", "Robotics Engineer", "Civil Engineer", 
        "Electronics Design Engineer", "Web Developer", "Quality Control Inspector", 
        "Video Game Tester", "Game Developer", "IT Project Manager", "Database Analyst", "Software Quality Assurance Tester", "Database Administrator"
    ],
    "Science and Research": [
        "Research Scientist", "Biologist", "Astronomer", "Environmental Scientist", 
        "Biotechnologist", "Zoologist", "Forensic Scientist", "Geologist", 
        "Wildlife Biologist", "Marine Biologist", "Biomedical Researcher"
    ],
    "Business and Finance": [
        "Accountant", "Financial Analyst", "Financial Planner", "Financial Auditor", 
        "Financial Advisor", "Investment Banker", "Tax Accountant", "Insurance Underwriter", "Product Manager"
    ],
    "Marketing and Sales": [
        "Salesperson", "Marketing Manager", "Marketing Coordinator", "Event Planner", 
        "Real Estate Agent", "Market Research Analyst", "Marketing Copywriter", 
        "Social Media Manager", "Advertising Executive", "Public Relations Specialist", 
        "Market Researcher", "Event Photographer", "Chef", "Marketing Analyst"
    ],
    "Human Resources and Administration": [
        "Human Resources Manager", "HR Recruiter", "Administrative Officer", 
        "IT Project Manager"
    ],
    "Design and Creative Arts": [
        "Graphic Designer", "Architect", "Artist", "Fashion Designer", 
        "Musician", "Interior Designer", "Film Director", "Fashion Stylist", "Technical Writer"
    ],
    "Education and Counseling": [
        "Teacher", "Psychologist", "Elementary School Teacher", "Social Worker", 
        "Marriage Counselor", "Speech Pathologist", "Sports Coach", 
        "Rehabilitation Counselor", "Career Counselor"
    ],
    "Law and Public Safety": [
        "Lawyer", "Police Detective", "Police Officer", "Human Rights Lawyer", 
        "Customs and Border Protection Officer", "Diplomat", "Foreign Service Officer", 
        "Tax Collector", "Air Traffic Controller", "Airline Pilot", "Journalist"
    ],
    "Urban Planning and Environment": [
        "Urban Planner", "Construction Engineer", "Environmental Scientist", 
        "Environmental Engineer", "Forestry Technician", "Wildlife Conservationist"
    ]
}


inverse_categories = {v: k for k, values in categories.items() for v in values}

df['Field'] = df['Career'].apply(lambda x: inverse_categories[x] if x in inverse_categories else 'Other Specialized Fields')

In [ ]:
df['Field'].unique()

In [ ]:
df['Field'].value_counts()

In [ ]:
# Codificación de la columna 'Career'
df['Career_encoded'] = LabelEncoder().fit_transform(df['Career'])
df['Field_encoded'] = LabelEncoder().fit_transform(df['Field'])

In [ ]:
df.corr()

In [ ]:
field_counts = df['Field'].value_counts()
sub_df = df[df['Field'].isin(field_counts[field_counts >= 7].index)]
sub_df.corr()

In [ ]:
sub_df

In [ ]:
word = "Physician"

result = model.most_similar(positive=[word])
result

In [ ]:
model.wv.get_vector(word)

In [ ]:
from sklearn import tree
from sklearn import svm

X = df.drop(['Career', 'Field', 'Career_encoded', 'Field_encoded'], axis=1)
Y = df['Field_encoded']

train_X = X[:int(0.8*len(X))]
train_Y = Y[:int(0.8*len(Y))]
test_X = X[int(0.8*len(X)):]
test_Y = Y[int(0.8*len(Y)):]

# model = tree.DecisionTreeClassifier()
model = svm.SVC()
model = model.fit(train_X, train_Y)

model.score(test_X, test_Y)


In [ ]:
model.get_params()